In [8]:
import numpy as np
import json

In [18]:
def distance_to(start:str, end:str, data):
    dis = data[start][end] 
    if (data[start][end] == "n/a"):
        return 0
    else:
        return dis

In [9]:
class Truck:
    def __init__(self, starting_depot) -> None:
        
        # Truck constants
        self.hourly_wage = 20
        self.overtime_wage = 30
        self.speed = 100
        self.cost_per_km = 0.68 + self.hourly_wage * self.speed / 100
        self.cost_per_km_overtime = 0.68 + self.overtime_wage * self.speed / 100
        
        # Truck variables
        self.home_depot = starting_depot
        self.current_city = None
        self.rest = False
        self.distance_driven_day = 0
        self.distance_driven_week = 0
        self.total_cost = 0
        self.packages = 0

        # To output
        self.output_dictionary = {
            "Total_cost": self.total_cost,
            "Monday": {
                
            }
        }

    def generate_day_info(self, day_distance:float, day_cost:float, day_deliveries:list):
        day_info = {
                "Total_distance": day_distance,
                "Total_cost": day_cost,
        }
        i = 1
        for delivery in day_deliveries:
            day_info[f'Delivery_{i}'] = delivery
            i += 1
        
    def generate_delivery_info(self, start_t, end_t, start_city, end_city, delivery_type, start_quant, end_quant):
        delivery_info = {
            "Start_time": start_t,
            "End_time": end_t,
            "Start_destination":start_city,
            "End_destination":end_city,
            "Distance": distance_to(start_city, end_city),
            "Delivery_type":delivery_type,
            "Start_quantity":start_quant,
            "End_quantity": end_quant,
        }
        return delivery_info


    def generate_output(self):
        pass

    def kill_trucker(self) -> None:
        self.rest = True

class City:
    def __init__(self, req_packages:int) -> None:
        self.req_packages = req_packages
        self.remaining_packages = 0
        self.depot = False

    def recieve_packages(self, packages) -> None:
        self.remaining_packages -= packages


In [10]:
#import distance data
with open(r'json/distance.json') as f:
    distance = json.load(f)

#import input data
with open(r'json/input.json') as f:
    input = json.load(f)

In [34]:
depots = ['Goderich', 'Toronto', 'Picton']
for city in distance.keys():
    lowest_dist = 1E9
    closest_depot = ""
    for depot in depots:
        dist = distance_to(city, depot, distance)
        
        if (dist != 0 and dist < lowest_dist):
            lowest_dist = dist
            closest_depot = depot

    print(f'{city} belongs to {closest_depot}')

Tilbury belongs to Goderich
Mississauga belongs to Toronto
Cornwall belongs to Picton
London belongs to Goderich
Windsor belongs to Goderich
Niagara_Falls belongs to Toronto
Barrie belongs to Toronto
Kingston belongs to Picton
Huntsville belongs to Toronto
North_Bay belongs to Toronto
Goderich belongs to Toronto
Toronto belongs to Picton
Picton belongs to Toronto
